# Fase 2: Extracción de Detalles de Películas con Selenium

WEBS
- IMDB: https://www.imdb.com/
- Rotten Tomatoes: https://www.rottentomatoes.com/

- Puntuacion de IMDB (en caso de que la tenga). ✅

- Puntuacion de Rotten Tomatoes (Tomatometer).

- Dirección (director/a o directore/as de cada película). ✅

- Gionistas (de cada película). ✅

- Argumento. ✅

- Duración (en minutos).

- Nombre de la película

In [7]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.by import By

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

#Imoprtamos librería para ver el % de avance
from tqdm import tqdm

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [8]:
lista_id = ['tt10579986', 'tt10597228', 'tt11097384', 'tt11152168', 'tt11237950', 'tt11822244', 'tt12053936', 'tt12193500', 'tt12255500', 'tt12767592', 'tt12861438', 'tt13036100', 'tt13433802', 'tt13561128', 'tt13576680', 'tt13695406', 'tt13965596', 'tt14117392', 'tt14256984', 'tt14257662', 'tt14478740', 'tt14564000', 'tt14643262', 'tt14866430', 'tt14904970', 'tt14978566', 'tt15005992', 'tt15239678', 'tt15427838', 'tt15535164', 'tt15572712', 'tt1592253', 'tt2368853', 'tt3663576', 'tt4392566', 'tt4601194', 'tt5834410', 'tt6398614', 'tt9371598', 'tt15791240', 'tt9134042', 'tt15769770', 'tt16139258', 'tt9810236', 'tt15721572', 'tt16867258', 'tt7131764', 'tt8547668', 'tt16913084', 'tt17037988', 'tt17073016', 'tt17590672', 'tt17731524', 'tt18024180', 'tt18213692', 'tt18556552', 'tt19336000', 'tt19368948', 'tt19898506', 'tt20407238', 'tt20411966', 'tt20453194', 'tt21065476', 'tt22022452', 'tt22394800', 'tt23461428', 'tt23747498', 'tt24176896', 'tt22062956', 'tt24948640', 'tt25785988', 'tt26238738', 'tt26439277', 'tt26446278', 'tt26731216', 'tt26731970', 'tt26768685', 'tt26909097', 'tt27006928', 'tt27205186', 'tt27248147', 'tt27439361', 'tt27459160', 'tt27478261', 'tt27487288', 'tt27496775', 'tt27533338', 'tt27534307', 'tt27607129', 'tt27626125', 'tt27635887', 'tt27746150', 'tt27750212', 'tt27814598', 'tt27819388', 'tt27827937',
'tt31912843', 'tt31913143', 'tt31913644', 'tt31913201', 'tt31913927', 'tt31918283', 'tt31918560', 'tt31923094', 'tt31924248', 'tt31925345', 'tt31937670', 'tt31937861', 'tt31938105', 'tt31939390', 'tt31939497', 'tt31940224', 'tt31940684', 'tt31945295', 'tt31945966', 'tt31947055', 'tt31947604', 'tt31947652', 'tt31948263', 'tt31949035', 'tt31955111', 'tt31955328', 'tt31955451', 'tt31955604', 'tt31956512', 'tt31956696', 'tt31956878', 'tt31957336', 'tt31960909', 'tt31961279', 'tt31961712', 'tt31963478', 'tt31963838', 'tt31968569', 'tt31968629', 'tt31969078', 'tt31969274', 'tt31970009', 'tt31969796', 'tt31970745', 'tt31971489', 'tt31972385', 'tt31972475', 'tt31972535', 'tt31973726', 'tt31973756', 'tt31974882', 'tt31975656', 'tt31975568', 'tt31985814', 'tt31985964', 'tt31986077', 'tt31986672', 'tt31987018', 'tt31987050', 'tt31987064', 'tt31996061', 'tt31997613', 'tt31999195', 'tt32011615', 'tt32012324', 'tt32012381', 'tt32014351', 'tt32019733', 'tt32020028', 'tt32020189', 'tt32020958', 'tt32022335', 'tt32023078', 'tt32023291', 'tt32023769', 'tt32026356', 'tt32027870', 'tt32029105', 'tt32029210', 'tt32035255', 'tt32035415', 'tt32045199', 'tt32045279', 'tt32046090', 'tt32046399', 'tt32047707', 'tt32050758', 'tt32055377', 'tt32055878', 'tt32056058', 'tt32056075', 'tt32056778', 'tt32058413', 'tt32045603', 'tt32056499', 'tt32058871', 'tt32045576', 'tt32058988', 'tt32058992', 'tt32059821', 'tt32060363', 'tt32062709', 'tt32062965', 'tt32063182', 'tt32063721', 'tt32064202', 'tt32065560', 'tt32066710', 'tt32066730', 'tt32081983', 'tt32082245', 'tt32082847', 'tt32083311', 'tt32083642', 'tt32085310', 'tt32085401', 'tt32085996', 'tt32086004', 'tt32085997', 'tt32086034', 'tt32086077', 'tt32086052', 'tt32089835', 'tt32094021', 'tt32094459', 'tt32094913', 'tt32095531', 'tt32096064', 'tt32097565', 'tt32099777', 'tt32104272', 'tt32104375', 'tt32104721', 'tt32104940', 'tt31722474', 'tt31727580', ...]

In [9]:
mini_lista = ['tt10579986',
 'tt10597228',
 'tt11097384',
 'tt11152168',
 'tt11237950']

# mini_lista = lista_id[0:10]

In [10]:
mini_lista

['tt10579986', 'tt10597228', 'tt11097384', 'tt11152168', 'tt11237950']

In [11]:
#Estas serían las listas extraídas de la Fase 1: del csv sacamos una lista con todos los títulos y otra con todos los años
lista_pelis_cortos=["Mom's Coming", 'Love Song and Power', "William Shakespeare's Romeo + Juliet"]
años_pelis_cortos=[2024, 2024, 1996]

#Unimos esas dos listas en un zip que utilizaremos para buscar dentro de Rotten Tomatoes
zip_pelis_cortos= list(zip(lista_pelis_cortos, años_pelis_cortos))

In [12]:
# creamos listas para almacenar la información obtetina de las webs
lista_punt_IMDB = []
lista_punt_tomato = []
lista_dire = []
lista_guionistas = []
lista_argumento = []
lista_duracion = []
lista_nombre_peli = []



In [13]:
# Inicializa el navegador Chrome
driver = webdriver.Chrome()

# Navega a la página web "http://www.imdb.com"
driver.get("http://www.imdb.com")

# Maximizar la ventana del navegador
driver.maximize_window()

# Acepta las cookies haciendo clic en el elemento con el selector CSS
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

#Iniciamos loop for para iterar en la lista de todas las películas
for peli in mini_lista:

    # Introducimos el número de ID de la peli en el buscador de la web
    driver.find_element('css selector', '#suggestion-search').send_keys(peli, Keys.ENTER)
    sleep(2)


    #Buscamos la información de las puntaciones de cada película
    #Empezamos un try/except loop per si hay películas que no tienen puntuación. En este caso appendearemos NULL en la lista
    try:
        lista_punt_IMDB.append(float(driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-1.iIQkEw > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK').text))
    except:
        lista_punt_IMDB.append('NULL')


    #Buscamos la información de los directorters de cada película  
    # Definimos los valores iniciales de nuestras variables para iniciar un loop while que serán:
    # un contador i y una lista vacía donde guardar todos los directores de ésta película
    i=0
    directores=[]
    #Iniciamos el loop while
    while True:  
        i+=1
        # Iniciamos try/expect loop para que cuando no tengamos más directores dé error y salganmos del loop while
        try:
            directores.append(driver.find_element('xpath', f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li[{i}]/a').text)
        except:
            break
    #Appendeamos la lista de directores de esta película en la lista general de directores
    if len(directores)!=0:
        lista_dire.append(directores)
    else:
        lista_dire.append('NULL')


    #Buscamos la información de los guionistas de cada película  
    # Definimos los valores iniciales de nuestras variables para iniciar un loop while que serán:
    # un contador i y una lista vacía donde guardar todos los guionistas de ésta película
    i=0
    guionistas=[]
    #Iniciamos el loop while
    while True:  
        i+=1
        # Iniciamos try/expect loop para que cuando no tengamos más guionistas dé error y salganmos del loop while
        try:
            guionistas.append(driver.find_element('xpath', f'/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div/ul/li[{i}]/a').text)
        except:
            break
    #Appendeamos la lista de guinistas de esta película en la lista general de guionistas
    if len(guionistas)!=0:
        lista_guionistas.append(guionistas)
    else:
        lista_guionistas.append('NULL')


    #Buscamos el argumento de cada película  
    #Empezamos un try/except loop per si hay películas que no tienen puntuación. En este caso appendearemos NULL en la lista
    try:
        lista_argumento.append(driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text)
    except:          
        lista_argumento.append('NULL')


    #Buscamos la duración de cada película en minutos 
    # Definimos las variables que necesitaremos
    # duracion = []
    #Creamos lista en la que guardaremos los dígitos de las horas y los minutos
    horas_y_minutos=[]

    # Iniciamos try/expect loop para que busque la sección. Si la encuenta la appendea en seccion_duracion, de lo contrario, appendea 'NULL'.
    try:
        duracion = driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ').text
    except:
        duracion = 'NULL'

    #De la sección, nos devuelve una lista con todos los campos. Hacemos un regex para extraer las horas y los minutos.
    #Buscamos el patrón "Xh Ymin": un número seguido de 'h', seguido de 0 o más espacios, seguido de otro número y 'min'
    patron = r'(\d+)h\s*(\d+)min|(\d+)h\s*(\d+)m'
    
    #Dentro de cada sección, buscamos por texto, que es cada parte de la sección (título, año, duración)
    #Buscamos el patrón definido dentro de cada parte de la sección, y lo guardamos en la variable 'match'
    match=re.search(patron, duracion)
    if match:
        #Si ha encontrado el patrón, guarda el primer grupo de captura (\d+) en la variable 'horas' y el segundo (\d+) en la variable 'minutos'
        horas=match.group(1)
        minutos=match.group(2)
        horas_y_minutos.append((int(horas), int(minutos)))
    else:
        #Si no existe la duración en la sección, appendeamos 'NULL'
        horas_y_minutos.append('NULL')

        
    #Ahora, el contenido de horas_y_minutos es [(2, 2), (1, 30), (1, 47), (1, 44), ('NULL)]. Vamos a pasarlo a minutos.
    #Creamos variable para guardar los minutos, de la que beberá la lista_duracion
    duracion_minutos=[]
    #Iteramos horas_y_minutos
    for tiempo in horas_y_minutos:
        #Si es 'NULL', appendeamos 'NULL'
        if tiempo == 'NULL':
            duracion_minutos.append('NULL')
        #Se puede dar el caso de que sólo haya un dígito, por ejemplo 2 (2 horas exactas). De ser así, multiplicamos las horas por 60.
        elif len(tiempo)==1:
            duracion_minutos.append(tiempo[0]*60)
        else:
            #Si hay más de un dígito, multiplicamos el primero por 60 y le sumamos los minutos.
            duracion_minutos.append((tiempo[0]*60) + (tiempo[1]))
        
    #Por último, appendeamos la lista final con los minutos totales
    lista_duracion.append(duracion_minutos)  
            
#Cerramos el navegador      
driver.close()   

In [14]:
#1. Entramos en la website y aceptamos cookies
driver=webdriver.Chrome()
driver.get('https://www.rottentomatoes.com/')
driver.maximize_window()
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
sleep(3)

#2. Iteramos el zip para extraer su puntuación
for titulo, año in tqdm(zip_pelis_cortos):

    #Intentamos buscar por el título; si existe entra en el primer resultado; de lo contrario, appenda un mensaje de 'NULL' (ponemos NULL para que las futuras consultas en SQL sean más prácticas)
    try: #bBuscamos la peli por título y año en la barra del buscador
        driver.find_element('css selector', '#header-main > search-algolia > search-algolia-controls > input').send_keys(f'{titulo} {año}', Keys.ENTER)
        sleep(3) #Hacemos click en el primer resultado
        driver.find_element('css selector', '#search-results > search-page-result:nth-child(2) > ul > search-page-media-row:nth-child(1) > a:nth-child(2)').click()
        sleep(3)
        try: #busca el tomatómetro y, si existe, lo appendea en la lista_punt_tomato
            lista_punt_tomato.append(driver.find_element('css selector', '#modules-wrap > div.media-scorecard.no-border > media-scorecard > rt-button:nth-child(3) > rt-text').text)
        except: #si no tiene tomatómetro, appendea 'NULL'
            lista_punt_tomato.append('NULL')
        sleep(3)
    except: #si la peli no existe devuelve mensaje 'NULL'
        lista_punt_tomato.append('NULL')

#Cerramos el navegador      
driver.close() 

100%|██████████| 3/3 [01:04<00:00, 21.34s/it]


## TESTEANDO 👇🏻

In [15]:
lista_punt_IMDB 


['NULL', 'NULL', 'NULL', 'NULL', 'NULL']

In [16]:
lista_punt_tomato 


['90%', 'NULL', '74%']

In [17]:
lista_dire 


[['Rishiking'],
 ['Erik Krefeld', 'Eddel Martinez'],
 ['Johan Renck'],
 ['John Krasinski'],
 ['Frederick Keeve']]

In [18]:
lista_guionistas 


[['Rishiking'],
 ['Stevie Kinchen', 'Erik Krefeld'],
 ['Jaroslav Kalfar', 'Colby Day'],
 ['John Krasinski'],
 ['Frederick Keeve']]

In [19]:
lista_argumento


["A soul allegedly with a 20-year-old girl as her daughter, whom she wants to take with her and the elder sister's wants to stop the soul from doing so.",
 "This is a story about finding the courage to be one's self.",
 'Half a year into his solo mission on the edge of the solar system, an astronaut concerned with the state of his life back on Earth is helped by an ancient creature he discovers in the bowels of his ship.',
 "A young girl who goes through a difficult experience begins to see everyone's imaginary friends who have been left behind as their real-life friends have grown up.",
 'Logline: Two newly engaged lovers return to Los Angeles after two years in New York. One of them, a piano accompanist, gets a job at the local ballet studio while attempting to resolve a family tragedy that still haunts him.']

In [20]:
lista_duracion 

[['NULL'], ['NULL'], ['NULL'], ['NULL'], ['NULL']]

In [ ]:
# Inicializa el navegador Chrome
driver = webdriver.Chrome()

# Navega a la página web "http://www.imdb.com"
driver.get("http://www.imdb.com")

# Maximizar la ventana del navegador
driver.maximize_window()

# Acepta las cookies haciendo clic en el elemento con el selector CSS
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

#Iniciamos loop for para iterar en la lista de todas las películas
for peli in mini_lista:
     # Iniciamos try/expect loop para que busque la sección. Si la encuenta la appendea en seccion_duracion, de lo contrario, appendea 'NULL'.
    # Introducimos el número de ID de la peli en el buscador de la web
    driver.find_element('css selector', '#suggestion-search').send_keys(peli, Keys.ENTER)
    sleep(2)
   
   
    try:
        seccion_duracion.append(driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ').text)
    except:
        seccion_duracion.append('NULL')
   
    print(seccion_duracion)

    if seccion_duracion == 'NULL':
        lista_duracion.append('NULL')

    else:

        patron = r'(\d+)h\s*(\d+)min|(\d+)h\s*(\d+)m'
        #Creamos lista en la que guardaremos los dígitos de las horas y los minutos
        horas_y_minutos=[]
        #Iteramos la lista de secciones que sacamos en el paso anterior
        for i in seccion_duracion:
            print(i)
            #Dentro de cada sección, buscamos por texto, que es cada parte de la sección (título, año, duración)
            #Buscamos el patrón definido dentro de cada parte de la sección, y lo guardamos en la variable 'match'
            match=re.search(patron, i)
        
            if match:
                #Si ha encontrado el patrón, guarda el primer grupo de captura (\d+) en la variable 'horas' y el segundo (\d+) en la variable 'minutos'
                horas=match.group(1)
                print(horas)
                minutos=match.group(2)
                print(minutos)
                horas_y_minutos.append((int(horas), int(minutos)))
                print(horas_y_minutos)
            else:
                #Si no existe la duración en la sección, appendeamos 'NULL'
                horas_y_minutos.append('NULL')

#Cerramos el navegador      
driver.close() 

['NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL']
NULL
NULL
NULL
NULL
NULL
NULL
['NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL']
NULL
NULL
NULL
NULL
NULL
NULL
NULL
['NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL']
NULL
NULL
NULL
NULL
NULL
NULL
NULL
NULL
['NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL']
NULL
NULL
NULL
NULL
NULL
NULL
NULL
NULL
NULL
['NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL']
NULL
NULL
NULL
NULL
NULL
NULL
NULL
NULL
NULL
NULL


In [ ]:
horas

In [ ]:
horas_y_minutos

['NULL', 'NULL', 'NULL', 'NULL', 'NULL']